In [1]:
import numpy as np
#import modin.pandas as pd
import pandas as pd
from ppca import PPCA
import six

import warnings

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation, NMF
from sklearn.cluster import KMeans, MiniBatchKMeans, AgglomerativeClustering, Birch, FeatureAgglomeration, SpectralClustering, SpectralCoclustering, SpectralBiclustering, DBSCAN, AffinityPropagation, MeanShift
from sklearn.mixture import GaussianMixture, BayesianGaussianMixture
from sklearn.decomposition import PCA, FastICA, IncrementalPCA, KernelPCA, FactorAnalysis, TruncatedSVD, SparsePCA, MiniBatchSparsePCA, DictionaryLearning, MiniBatchDictionaryLearning
from sklearn.manifold import TSNE, MDS, SpectralEmbedding, LocallyLinearEmbedding, Isomap
from umap import UMAP
from flair.embeddings import WordEmbeddings, CharacterEmbeddings, StackedEmbeddings, FlairEmbeddings, BertEmbeddings, ELMoEmbeddings, DocumentPoolEmbeddings
from flair.data import Sentence

import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

global HYPERTOOLS_NOTEBOOK_MODE; HYPERTOOLS_NOTEBOOK_MODE = False

#import plotly_express as px

In [57]:
#debugging stuff
from matplotlib import pyplot as plt
import seaborn as sns

%matplotlib inline

In [2]:
def hypertools_notebook_mode(toggle=None):
    global HYPERTOOLS_NOTEBOOK_MODE
    
    if toggle:
        print('Enabling inline hypertools plots')
        HYPERTOOLS_NOTEBOOK_MODE = True
    else:
        print('Disabling interactive hypertools plots')
        HYPERTOOLS_NOTEBOOK_MODE = False
    return None

In [3]:
hypertools_notebook_mode(True)

Enabling inline hypertools plots


In [4]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL']='2'
import tensorflow.compat.v1 as tf
#import tensorflow as tf
import tensorflow_hub as hub

tf.disable_v2_behavior()
tf.disable_eager_execution()

2019-05-28 16:27:04,023 From /usr/local/lib/python3.6/site-packages/tensorflow/python/compat/v2_compat.py:63: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
Instructions for updating:
non-resource variables are not supported in the long term


In [5]:
from srm import SRM
from procrustes import procrustes

#working from align.py:
# - include SRM, hyperalignment, and procrustes as options (for procrustes, align everything to the first observation)
#   - modify these functions to return both the transformed data and also a function that will map a new set of observations into the common space
# - format the data to ensure it's a list of dataframes
# - carve out hyperalignment as a separate function (like SRM)
# - use the indices to create a common set of dataframes, all with indices from the full set (union) of indices across all dataframes in the list, and all with the same number of columns (zero-padding to match the dataframe with the most columns).  compute the transformations
#   using just the common indices (intersection).  But the transform function should work with arbitrary data, including from non-matching indices.
#   The full align function should return a list of dataframes of the same size/shape as the inputted list, and where each dataframe has the same number of rows (at the same indices)
#   as the corresponding dataframes in the original list, but every dataframe in the aligned list has the same number of columns.

#consider: returnning a new sub-class of dataframe that includes the inverse transformation functions, allowing the data to be transformed back into the original data spaces
#in turn, this would return a new TransformableDataFrame that could map the (original) data back into the transformed space.

In [6]:
from configparser import ConfigParser
#from collections import defaultdict

In [7]:
defaults = ConfigParser()
defaults.read('defaults.ini');

In [8]:
init_notebook_mode(connected=True)

In [9]:
reduce_models = ['DictionaryLearning', 'FactorAnalysis', 'FastICA', 'IncrementalPCA', 'KernelPCA', 'LatentDirichletAllocation', 'MiniBatchDictionaryLearning',
 'MiniBatchSparsePCA', 'NMF', 'PCA', 'SparseCoder', 'SparsePCA', 'TruncatedSVD', 'UMAP', 'TSNE', 'MDS', 'SpectralEmbedding', 'LocallyLinearEmbedding', 'Isomap']
cluster_models = ['AffinityPropagation', 'AgglomerativeClustering', 'Birch', 'DBSCAN', 'FeatureAgglomeration', 'KMeans', 'MeanShift', 'MiniBatchKMeans', 'SpectralBiclustering', 'SpectralClustering', 'SpectralCoclustering', 'DBSCAN', 'AffinityPropagation', 'MeanShift']
mixture_models = ['GaussianMixture', 'BayesianGaussianMixture', 'LatentDirichletAllocation', 'NMF']
decomposition_models = ['LatentDirichletAllocation', 'NMF']
text_vectorizers = ['CountVectorizer', 'TfidfVectorizer']
interpolation_models = ['linear', 'time', 'index', 'pad', 'nearest', 'zero', 'slinear', 'quadratic', 'cubic', 'spline', 'barycentric', 'polynomial']
text_models = ['universal_sentence_encoder', 'LatentDirichletAllocation', 'NMF']
corpora = ['wiki', 'nips', 'sotus']


### API ideas

1. [x] Write a single function to format data into a list of pandas dataframes:
    - [x] If text, should vectorize text according to default (or given) models.  Allow user to (optionally) pass in row/column labels for the text data.  Each string gets one row; a list of strings yields a len(list) by n-features dataframe
    - [x] If numpy arrays, set indices to row count and columns to column count
    - [x] If pandas dataframe, keep as is but turn into a list
    - [x] If only a single matrix is given, still return a list
    - [x] Dimensions may be mismatched
    - [x] Missing data is left unchanged
2. [x] Write a function to take a list of dataframes and use PPCA to fill in missing values.  Also use interpolation to fill in missing rows
3. [x] Write a reduce function that can work in batch mode (reduce a single dataframe or a list of dataframes)
4. [x] Write a cluster function that can work in batch mode (return cluster labels or mixture proportions for a single dataframe or a list of dataframes)
5. Write a text-vectorizer function that wraps universal sentence encoder, LDA, NMF, etc.
6. Write an align function that aligns all dataframes in a list into a common space
7. Write a normalize function that z-scores the data
8. Write a plot function that takes a list of dataframes and plots them
    - Need a mat2colors function that colorizes data points according to group labels, cluster assignments, mixture proportions, or user-specified matrices.  Take in a colormap as an argument
    - Need support for multicolored lines in addition to per-observation marker colors.  For 3d plots, could use streamtubes...but for 2d plots this won't work, so maybe a segment solution would be appropriate.
    - Potentially allow for different line/marker colors
    - Support for animations:
      - Spin
      - Sliding window
      - Sliding window with tail
      - Sliding window with head
    - Support for multiindex dataframes (hierarchical plots with different coloring/line thickness)
9. Write a hyper-function that takes in a "messy" dataset, applies formatting, PPCA, reduce, alignment, and clustering (as specified) and returns a formatted/processed dataset.
10. Add hyper-function call to every helper function, so that all analyses can be applied from any call to any other function

In [10]:
def is_dataframe(x):
    return type(x).__module__ in ['pandas.core.frame', 'modin.pandas.dataframe']

def is_multiindex_dataframe(x):
    return is_dataframe(x) and (type(x.index).__module__ == 'pandas.core.indexes.multi')

def is_array(x):
    return (not ('str' in str(type(x)))) and (type(x).__module__ == 'numpy')

def is_empty(x):
    return (x is None) or (len(x) == 0)

def is_text(x):
    if type(x) == list:
        return np.all([is_text(t) for t in x])
    return (type(x) in six.string_types) or (type(x) == np.str_)

In [11]:
def to_str_list(x, encoding='utf-8'):
    def to_string(s):
        if type(s) == str:
            return s
        elif is_empty(s) or (s == None):
            return ''
        elif type(s) in [bytes, np.bytes_]:
            return s.decode(encoding)
        elif is_array(s) or is_dataframe(s) or (type(s) == list):
            if len(s) == 1:
                return to_string(s[0])
            else:
                return to_str_list(s, encoding=encoding)
        else:
            return str(s)
    
    if is_array(x) or (type(x) == list):        
        return [to_string(s) for s in x]
    elif is_text(x):
        return [x]
    else:
        raise Exception('Unsupported data type: {type(x)}')

In [12]:
def get_corpus(c, encoding='utf-8'):
    import hypertools as hyp
    if c in corpora:
        fname = os.path.join(eval(defaults['data']['datadir']),'corpora', f'{c}.npy')
        if not os.path.exists(fname):
            if not os.path.exists(os.path.abspath(os.path.join(fname, os.pardir))):
                os.makedirs(os.path.abspath(os.path.join(fname, os.pardir)))
            corpus_words = to_str_list(hyp.load(c).data[0]) #TODO: FIX THIS TO NOT CALL HYPERTOOLS!
            
            np.save(fname, corpus_words)
            return corpus_words
        else:
            corpus_words = np.load(fname, allow_pickle=True)
            return to_str_list(corpus_words)
    else:
        if is_text(c):
            if type(c) == list:
                return c
            else:
                return [c]
        elif os.path.exists(c):
            return to_str_list([x[0] for x in np.load(c, allow_pickle=True).tolist()])
        else:
            raise Exception(f'Unknown corpus: {c}')

In [13]:
def vectorize_text(text, vectorizer='CountVectorizer', vocabulary=None, return_vocab=False):
    if not (type(text) == list):
        text = [text]
    assert is_text(text), f'Must vectorize a string or list of strings (given: {type(text)})'
    
    if type(vectorizer) in six.string_types:
        assert vectorizer in text_vectorizers, f'Text vectorizer must be a function or a member of {text_vectorizers}'
        vectorizer = eval(vectorizer)
    assert callable(vectorizer), f'Text vectorizer must be a function or a member of {text_vectorizers}'
    
    text2vec = vectorizer(max_df=eval(defaults['text']['max_df']),
                          min_df=eval(defaults['text']['min_df']),
                          stop_words=defaults['text']['stop_words'],
                          strip_accents=defaults['text']['strip_accents'],
                          lowercase=eval(defaults['text']['lowercase']),
                          vocabulary=vocabulary)
    vectorized_text = text2vec.fit_transform(text)
    
    if not return_vocab:
        return vectorized_text
    else:
        vocab = text2vec.get_feature_names()
        return vectorized_text, vocab

In [65]:
def get_text_model(corpus, model, vectorizer, n_components=50):
    if type(model) in six.string_types:
        assert model in text_models, f'Text model must be a function or a member of {text_models}'
        model = eval(model)
    assert callable(model), f'Text model must be a function or a member of {text_models}'
    
    if type(vectorizer) in six.string_types:
        assert vectorizer in text_vectorizers, f'Text vectorizer must be a function or a member of {text_vectorizers}'
        vectorizer = eval(vectorizer)
    assert callable(vectorizer), f'Text vectorizer must be a function or a member of {text_vectorizers}'
    
    if corpus in corpora:
        saveable = True
    else:
        if not os.path.exists(corpus):        
            assert is_text(corpus), f'Corpus must be a list of strings, or one of {corpora}'
        saveable = False
    
    if saveable:
        fname = os.path.join(eval(defaults['data']['datadir']),'text-models', model.__name__, f'{corpus}-{vectorizer.__name__}-{n_components}.npz')    
        if not os.path.exists(os.path.abspath(os.path.join(fname, os.pardir))):
            os.makedirs(os.path.abspath(os.path.join(fname, os.pardir)))
    
    if saveable and os.path.exists(fname):
        with np.load(fname, allow_pickle=True) as x:
            return {'vocab': x['vocab'].tolist(), 'model': x['model'].tolist()}
    else:
        corpus = get_corpus(corpus)
        vectorized_corpus, vocab = vectorize_text(corpus, vectorizer=vectorizer, return_vocab=True)
        
        if n_components == None:
            n_components = eval(defaults['text']['topics'])
        args = {'n_components': n_components,
                'max_iter': eval(defaults['text']['max_iter'])}
        
        if model.__name__ == 'NMF' and (args['n_components'] > len(corpus)):
            args['n_components'] = len(corpus)
        
        if model.__name__ == 'LatentDirichletAllocation':
            args['learning_method'] = defaults['text']['learning_method']
            args['learning_offset'] = eval(defaults['text']['learning_offset'])
        
        #return args, vectorized_corpus, vocab
        
        embeddings = model(**args).fit(vectorized_corpus)
        
        if saveable:
            np.savez(fname, vocab=vocab, model=embeddings)
        
        return {'vocab': vocab, 'model': embeddings}        

In [68]:
wiki_NMF_model = get_text_model('wiki', 'NMF', 'CountVectorizer')

In [69]:
wiki_LDA_model = get_text_model('wiki', 'LatentDirichletAllocation', 'CountVectorizer')

In [70]:
nips_NMF_model = get_text_model('nips', 'NMF', 'CountVectorizer')

In [71]:
nips_LDA_model = get_text_model('nips', 'LatentDirichletAllocation', 'CountVectorizer')

In [72]:
sotus_NMF_model = get_text_model('sotus', 'NMF', 'CountVectorizer')

In [73]:
sotus_LDA_model = get_text_model('sotus', 'LatentDirichletAllocation', 'CountVectorizer')

In [95]:
def text_vectorizer(text, model='universal_sentence_encoder', **kwargs):
    def universal_sentence_encoder(text, **kwargs):        
        if 'USE_corpus' in kwargs.keys():
            corpus = kwargs['USE_corpus']
        else:
            corpus = defaults['text']['USE_corpus']
            #if not os.path.exists(corpus):
            #    os.system(f"'https://tfhub.dev/google/universal-sentence-encoder-large/3?tf-hub-format=compressed' | tar -zxvC {defaults['text']['USE_corpus']}")
        
        assert os.path.exists(corpus), f'Corpus not found: {corpus}'

        encoder = hub.Module(corpus)

        tf.logging.set_verbosity(tf.logging.ERROR)
        with tf.Session() as session:
            session.run([tf.global_variables_initializer(), tf.tables_initializer()])        
            return session.run(encoder(text))
    
    def sklearn_vectorizer(text, model, **kwargs):                
        if 'corpus' in kwargs.keys():
            corpus = kwargs['corpus']
        else:
            corpus = defaults['text']['corpus']            
        
        assert (corpus in corpora) or is_text(corpus) or os.path.exists(corpus), f'Cannot use corpus: {corpus}'
        
        if 'vectorizer' in kwargs.keys():
            vecterizer = kwargs['vectorizer']
            kwargs.pop('vectorizer', None)
        else:
            vectorizer = defaults['text']['vectorizer']
        
        model = get_text_model(corpus, model, vectorizer)
        return model['model'].transform(vectorize_text(text, vectorizer=vectorizer, vocabulary=model['vocab']))
    
    assert (model in text_models) or (callable(model)), f'Unsupported model: {model}'
    if not (type(text) == list):
        text = [text]
    
    if callable(model):
        return model(text, **kwargs)
    elif model == 'universal_sentence_encoder':
        return universal_sentence_encoder(text, **kwargs)
    else:
        return sklearn_vectorizer(text, model, **kwargs)
    

In [102]:
def panda_stack(data, force=True, names=None, keys=None, verify_integrity=False, sort=False, copy=True, ignore_index=False, levels=None, **kwargs):
    '''
    Take a list of DataFrames with the same number of columns and (optionally)
    a list of names (of the same length as the original list; default:
    range(len(x))).  Return a single MultiIndex DataFrame where the original
    DataFrames are stacked vertically, with the data names as their level 1
    indices and their original indices as their level 2 indices.
    
    INPUTS
    data: data in any format (text, numpy arrays, pandas dataframes, or a mixed list (or nested lists) of those types)
    text_vectorizer: function that takes a string (or list of strings) and returns a numpy array or dataframe.  If
    force is False, must pass in a list of DataFrames.
    
    force: if True, use format_data to coerce everything into a list of pandas dataframes.
    
    text_vectorizer: function for turning text data into DataFrames, used if force is True
    
    Also takes all keyword arguments from pandas.concat except axis, join, join_axes
    
    All other keywork arguments (if any) are passed to text_vectorizer
    
    OUTPUTS
    a single MultiIndex DataFrame
    '''
    
    if force:
        data = format_data(data, **kwargs)
    
    assert type(data) == list, 'Must either pass a list of DataFrames or set force to True'
    assert np.all([is_dataframe(d) for d in data]), 'Must either pass a list of DataFrames or set force to True'
    assert len(np.unique([d.shape[1] for d in data])) == 1, 'All DataFrames must have the same number of columns'    
    template = data[0].columns.values
    for i, d1 in enumerate(data):
        template = d1.columns.values
        for d2 in data[(i+1):]:
            assert np.all([(c in template) for c in d2.columns.values]), 'All DataFrames must have the same columns'
    
    if keys is None:
        keys = np.arange(len(data), dtype=np.int)
    
    assert is_array(keys) or (type(keys) == list), f'keys must be None or a list or array of length len(data)'
    assert len(keys) == len(data), f'keys must be None or a list or array of length len(data)'
    
    return pd.concat(data, axis=0, join='outer', join_axes=None, names=names, keys=keys, 
                     verify_integrity=verify_integrity, sort=sort, copy=copy, ignore_index=ignore_index, levels=levels)

In [103]:
def panda_unstack(x):
    if not is_multiindex_dataframe(x):
        if is_dataframe(x):
            return x
        else:
            raise Exception(f'Unsupported datatype: {type(x)}')
    
    names = list(x.index.names)
    grouper = 'ID'
    if not (grouper in names):
        names[0] = grouper
    elif not (names[0] == grouper):
        for i in np.arange(len(names)): #trying n things other than 'ID'; at least one of them must be outside of the n-1 remaining names
            next_grouper = f'{grouper}{i}'
            if not (next_grouper in names):
                names[0] = next_grouper
                grouper = next_grouper
                break
    assert names[0] == grouper, 'Unstacking error'
    
    x.index.rename(names, inplace=True)    
    unstacked = [d[1].set_index(d[1].index.get_level_values(1)) for d in list(x.groupby(grouper))]
    if len(unstacked) == 1:
        return unstacked[0]
    else:
        return unstacked

In [104]:
def format_data(x, **kwargs):
    '''
    INPUTS
    x: data in any format (text, numpy arrays, pandas dataframes, or a mixed list (or nested lists) of those types)
    text_vectorizer: function that takes a string (or list of strings) and returns a numpy array or dataframe
    text_kwargs: dictionary of keywork arguments to pass to text_vectorizer
    
    OUTPUTS
    a list of pandas dataframes
    '''
    
    def to_dataframe(y):
        if is_dataframe(y):
            return y
        elif is_array(y):
            assert np.ndim(y) == 2, 'all data matrices must be 2-dimensional'
            idx = np.arange(y.shape[0])
            cols = np.arange(y.shape[1])
            return pd.DataFrame(data=y, index=idx, columns=cols)
        elif type(y) == list:
            return [to_dataframe(i) for i in y]
        elif is_text(y):
            if 'text_model' in kwargs.keys():
                text_model = kwargs['text_model']
            else:
                text_model = defaults['text']['model']
                
            if 'text_kwargs' in kwargs.keys():
                text_kwargs = kwargs['text_kwargs']
            else:
                text_kwargs = {}
            return to_dataframe(text_vectorizer(y, model=text_model, **text_kwargs))
        else:
            raise NotImplementedError(f'Unsupported data type: {type(y)}')
    
    if type(x) == list:
        return [to_dataframe(i) for i in x]
    else:
        return [to_dataframe(x)]

In [105]:
def fill_missing(x, apply_ppca=True, interpolation='linear', **kwargs):
    if type(x) == list:
        return [fill_missing(i, apply_ppca=apply_ppca, interpolation=interpolation, **kwargs) for i in x]
    elif is_array(x):
        return fill_missing(pd.DataFrame(x), apply_ppca=apply_ppca, interpolation=interpolation, **kwargs).values
    elif not is_dataframe(x):
        raise Exception('Unsupported datatype: f{type(x)}')
    
    if 'method' in interpolate_kwargs.keys():
        warnings.warn(f"Overloaded keyword argument; ignoring interpolation parameter ({interpolation}) in favor of keyword-specified method: {interpolate_kwargs['method']}")
    else:
        interpolate_kwargs['method'] = interpolation
    
    assert interpolate_kwargs['method'] in interpolation_models, f"Unsupported interpolation type: '{interpolate_kwargs['method']}'"
    
    if apply_ppca:
        covariance_model = PPCA()
        covariance_model.fit(x.values)
        x.values = covariance_model.transform()
    
    if not ('inplace' in kwargs.keys()):
        kwargs['inplace'] = False
    
    if not ('axis' in kwargs.keys()):
        kwargs['axis'] = 0
        
    if 'method' in kwargs.keys():
        warnings.warn(f"Overloaded keyword argument; ignoring interpolation parameter ({interpolation}) in favor of keyword-specified method: {kwargs['method']}")
    else:
        kwargs['method'] = interpolation
    
    return x.interpolate(**kwargs)

In [106]:
def reduce(data, algorithm='IncrementalPCA', stack=False, keys=None, ndims=3, fillna=False, interpolation_kwargs={}, **kwargs):
    '''
    ARGUMENTS:
    data: data to reduce (numpy array or compatable, or a pandas
          dataframe or compatable).  Formatted as a 2d matrix whose
          rows are observations and whose columns are feature
          dimensions.
    
    algorithm: one of: 'DictionaryLearning', 'FactorAnalysis',
          'FastICA', 'IncrementalPCA', 'KernelPCA',
          'LatentDirichletAllocation', 'MiniBatchDictionaryLearning',
          'MiniBatchSparsePCA', 'NMF', 'PCA', 'SparseCoder',
          'SparsePCA', 'TruncatedSVD', or 'UMAP'.  Calls the
          relevant scikit-learn or UMAP function.  Can also
          pass a function directly.
    
    ndims: the number of dimensions (columns) in the result (default: 3).
          if ndims > data.shape[1], the right-most columns will be
          zero-padded.
    
    fillna: if True, use PPCA and interpolation to fill in nan-valued entries (default: False)
    
    interpolation_kwargs: keyword arguments passed to the interpolation function, used
           when fillna is True.
    
    stack: if True, create a single (stacked) MultiIndex DataFrame out of
           the inputted data list and return a version of the same dataframe,
           but with ndims columns (named range(ndims))
    
    keys: a name for each data matrix (default: None; name each set of observations
          range(len(data))).  Only relevant when stack is True.
    
    all additional keyword arguments are passed to the reduce function
    
    RETURNS:
    pandas dataframe (or list of dataframes) with number-of-observations rows and
    ndims columns
    '''
    
    if type(algorithm) == str:
        assert algorithm in reduce_models, f'Unknown algorithm: {algorithm}' #security check to prevent executing arbitrary code
        algorithm = eval(algorithm)
    else:
        if not (('sklearn.decomposition' in algorithm.__module__) or ('umap.umap_' in algorithm.__module__)):
            raise Exception(f'Unknown algorithm: {algorithm}')
    
    assert ndims >= 0, f'Number of dimensions must be non-negative (given: {ndims})'
    
    warnings.filterwarnings('ignore', category=RuntimeWarning)
    
    data = format_data(data)
    
    if fillna:
        data = fill_missing(data, **interpolation_kwargs)
    
    stacked_data = panda_stack(data, keys=keys)
    vals = stacked_data.values
    
    if vals.shape[1] > ndims:
        model = algorithm(n_components=ndims, **kwargs)
        reduced_vals = model.fit_transform(vals)
    elif vals.shape[1] < ndims:
        reduced_vals = np.concatenate([vals, np.zeros([vals.shape[0], ndims-vals.shape[1]])], axis=1)
    else:
        reduced_vals = vals
    
    reduced_data = pd.DataFrame(data=reduced_vals, index=stacked_data.index, columns=np.arange(ndims))
    if stack:
        return reduced_data
    else:
        return panda_unstack(reduced_data)

In [107]:
def cluster(data, algorithm='KMeans', k=10, stack=False, keys=None, fillna=False, interpolation_kwargs={}, return_model=False, **kwargs):
    if type(algorithm) == str:
        assert (algorithm in cluster_models) or (algorithm in mixture_models), f'Unknown algorithm: {algorithm}' #security check to prevent executing arbitrary code
        algorithm = eval(algorithm)
    else:
        if not (('sklearn.cluster' in algorithm.__module__) or ('sklearn.decomposition' in algorithm.__module__) or ('sklearn.mixture' in algorithm.__module__)):
            raise Exception(f'Unknown algorithm: {algorithm}')
    
    k_dict = {'cluster_models': 'n_clusters', 'mixture_models': 'n_components'}
    for model_type in k_dict.keys():
        if algorithm.__name__ in eval(model_type):
            if not (k_dict[model_type] in kwargs.keys()):
                kwargs[k_dict[model_type]] = k
            else:
                warnings.warn(f"Overloaded keyword argument; ignoring k value ({k}) in favor of keyword-specified {k_dict[model_type]}: {kwargs[k_dict[model_type]]}")
            break
    
    if not ('verbose' in kwargs.keys()):
        kwargs['verbose'] = eval(defaults['cluster']['verbose'])
    
    data = format_data(data)
    
    if fillna:
        data = fill_missing(data, **interpolation_kwargs)
    
    stacked_data = panda_stack(data, keys=keys)
    vals = stacked_data.values
    
    if algorithm.__name__ in decomposition_models:
        if np.min(vals) < 0:
            vals -= np.min(vals) #avoid negative values by setting minimum value to 0
    
    model = algorithm(**kwargs)
    model.fit(vals)
    
    if hasattr(model, 'n_clusters'):
        n = 1
    elif hasattr(model, 'n_components'):
        n = model.n_components
    else:
        raise NotImplementedError(f'Unsupported model: {model}')
    
    if hasattr(model, 'predict_proba'):
        labels = model.predict_proba(vals)
    elif hasattr(model, 'predict'):
        labels = model.predict(vals)
    elif hasattr(model, 'transform'):
        labels = model.transform(vals)
    else:
        raise NotImplementedError(f'Unsupported model: {model}')
    
    labels = np.array(labels, ndmin=2)
    if n == 1:
        labels = labels.T
    
    stacked_labels = pd.DataFrame(data=labels, index=stacked_data.index, columns=np.arange(n))
    
    if stack:
        if return_model:
            return stacked_labels, model
        else:
            return stacked_labels
    else:
        if return_model:
            return panda_unstack(stacked_labels), model
        else:
            return panda_unstack(stacked_labels)

In [108]:
xs = [np.cumsum(np.random.randn(100, 10), axis=0) for i in np.arange(10)]

In [109]:
x_stacked = panda_stack(xs)

In [110]:
cluster(xs, stack=True, algorithm='GaussianMixture', k=5)

0              1             2             3              4
0 0   9.206771e-32   2.982723e-08  1.000000e+00  3.981482e-15   6.785754e-65
  1   2.379496e-38   5.022306e-09  1.000000e+00  3.148426e-16   4.031204e-69
  2   4.178504e-30   9.408703e-10  1.000000e+00  8.460444e-14   1.914125e-76
  3   4.590397e-35   1.058299e-08  1.000000e+00  6.257286e-17   1.202876e-73
  4   3.068630e-42   3.495378e-07  9.999997e-01  3.341990e-19   1.765552e-76
  5   1.368831e-37   9.776744e-08  9.999999e-01  5.720108e-17   4.728701e-74
  6   3.407189e-28   1.565482e-07  9.999998e-01  9.295124e-13   4.951995e-74
  7   8.479001e-34   6.315589e-09  1.000000e+00  1.008498e-11   1.019035e-76
  8   7.286520e-30   4.291881e-09  1.000000e+00  5.473085e-11   3.685868e-75
  9   1.310345e-38   4.020416e-12  1.000000e+00  2.472823e-11   1.249698e-63
  10  1.359102e-46   4.566061e-10  1.000000e+00  2.627805e-12   2.671041e-57
  11  1.843655e-41   3.044659e-12  1.000000e+00  2.025596e-12   7.100379e-62
  12  2.562205e-38   2.334900e-11  1.000000e+00  7.698940e-13   1.175900e-64
  13  6.729498e-32   7.210162e-09  1.000000e+00  1.955707e-12   9.322263e-64
  14  7.703805e-31   1.085468e-09  1.000000e+00  1.857807e-14   2.431265e-68
  15  4.275578e-26   4.674370e-09  1.000000e+00  1.170602e-13   1.218697e-74
  16  1.888178e-29   3.974863e-11  1.000000e+00  6.495855e-19   3.058667e-78
  17  4.824028e-33   1.234923e-10  1.000000e+00  9.254902e-20   1.440489e-76
  18  5.367847e-34   3.032281e-12  1.000000e+00  1.681660e-23   1.473309e-72
  19  5.825050e-36   1.527632e-10  1.000000e+00  4.478558e-24   3.180121e-80
  20  5.245003e-47   2.698106e-09  1.000000e+00  1.311379e-28   1.849608e-91
  21  1.503788e-54   2.883945e-11  1.000000e+00  2.771360e-32   5.401254e-89
  22  5.774988e-48   1.635135e-09  1.000000e+00  5.260600e-27   5.229449e-83
  23  5.581848e-52   4.766012e-08  1.000000e+00  6.770299e-32   1.526262e-83
  24  1.117416e-48   6.016758e-10  1.000000e+00  2.634148e-31   7.032925e-85
  25  3.010805e-45   4.884662e-13  1.000000e+00  8.507350e-31   4.062347e-89
  26  5.258428e-37   2.095142e-14  1.000000e+00  2.321652e-28   1.039659e-80
  27  1.988377e-40   3.123923e-13  1.000000e+00  4.987370e-30   1.429695e-80
  28  9.118237e-34   1.118229e-14  1.000000e+00  1.009445e-25   9.066386e-87
  29  6.046438e-36   1.006675e-15  1.000000e+00  3.230289e-28   3.877444e-96
...            ...            ...           ...           ...            ...
9 70  1.000000e+00   2.916344e-89  4.476361e-16  4.519504e-22  7.331381e-221
  71  1.000000e+00  4.503857e-100  9.106078e-19  5.369673e-23  1.029268e-225
  72  1.000000e+00  2.441616e-106  1.335421e-19  2.637402e-27  2.406582e-238
  73  1.000000e+00   2.467919e-99  4.690286e-18  8.535751e-26  8.481357e-239
  74  1.000000e+00  8.794096e-105  2.164849e-17  5.511255e-24  1.177689e-235
  75  1.000000e+00   1.967749e-94  7.204498e-17  2.124175e-19  7.968855e-239
  76  1.000000e+00  2.060187e-108  4.754285e-17  1.599434e-17  3.143968e-250
  77  1.000000e+00  1.276340e-127  7.954171e-18  8.449955e-22  8.588022e-274
  78  1.000000e+00  7.063929e-125  6.323835e-17  9.047459e-24  1.893580e-271
  79  1.000000e+00  3.352909e-118  1.934789e-16  4.828864e-18  2.521587e-257
  80  1.000000e+00  2.116656e-100  1.992280e-16  1.810920e-22  2.216416e-255
  81  1.000000e+00   3.655798e-95  1.433461e-18  1.657558e-26  4.088135e-239
  82  1.000000e+00   1.888670e-85  5.440781e-19  1.374347e-26  3.426350e-242
  83  1.000000e+00   1.636999e-86  1.709899e-19  1.067471e-28  1.714520e-228
  84  1.000000e+00  1.180887e-100  3.954387e-21  5.707242e-33  2.641718e-251
  85  1.000000e+00   7.517714e-96  1.340549e-20  2.832733e-36  9.081024e-267
  86  1.000000e+00   7.417636e-96  4.972116e-22  1.508341e-33  5.243890e-252
  87  1.000000e+00  5.753260e-105  4.252059e-25  7.134303e-36  2.224495e-260
  88  1.000000e+00  2.541874e-100  1.623785e-26  1.247922e-39  2.399254e-257
  89  1.000000e+00   1.389356e-96  2.050427e-27  3.847843e-49  2.997202e-283
  90  1.00000

In [111]:
from plotly.offline import plot as pyplot

In [112]:
def plotter(data, style='line', reduce_kwargs={}, **kwargs):
    global HYPERTOOLS_NOTEBOOK_MODE
        
    def combo_merge(a, b):
        combos = []
        for i in a:
            for j in b:
                if (len(i) <= 2):
                    if (len(j) <= 2):
                        combos.append(i+j)
                elif len(j) > 1:
                    combos.append(i + '+' + j)
        return combos
    
    marker_styles = ['.', 'o', 'scatter', 'marker', 'markers', 'bigmarker', 'bigmarkers']
    line_styles = ['-', '--', ':', '-:', 'line', 'lines']
    combo_styles = combo_merge(marker_styles, line_styles) + combo_merge(line_styles, marker_styles)
    big_markers = ['o', 'big']
    dash_styles = {'--': 'dash', '-:': 'dashdot', ':': 'dot'}
    
    def substr_list(style, x):
        '''
        style: a style string
        x: a list of substrings
        
        return: true if any of the strings in x is a substring of s, and false othewise    
        '''
        inds = np.array([s in style for s in x])
        if np.any(inds):
            return x[np.where(inds)[0][0]]
        else:
            return ''
    
    is_line = lambda s: substr_list(s, line_styles + combo_styles)
    is_marker = lambda s: substr_list(s, marker_styles + combo_styles)
    is_combo = lambda s: substr_list(s, combo_styles)
    
    is_dashed = lambda s: substr_list(s, list(dash_styles.keys()))
    is_bigmarker = lambda s: substr_list(s, big_markers)
    
    reduced_data = reduce(data, stack=True, **reduce_kwargs)
    split_data = panda_unstack(reduced_data)
    
    if reduced_data.shape[1] == 2:
        suffix = ''
    elif reduced_data.shape[1] == 3:
        suffix = '3D'
    else:
        raise NotImplementedError(f'Unsupported number of dimensions; reduced data must be 2d or 3d, given: {reduced_data.shape[1]}')
    
    #TODO: add support for multiple styles (up to one per line)
    #TODO: load in custom plot theme/style from defaults, also allow user to customize
    #TODO: draw bounding box around space
    #TODO: support animation:
    # - camera rotation
    # - window length around current timepoint
    # - opacity of prior timepoints
    # - opacity of current window
    # - opacity of future timepoints    
    #TODO: legend customization
    #TODO: label each point with its original values and/or a user-specified label
    #TODO: support per-datapoint colors, including for lines
    #TODO: support for mixture-based colors-- if 1d, map onto colormap.  If 1 < n-colors <= 3, pick
    #      a color for each dimension and then compute each coordinate as a weighted blend.  If > 3,
    #      map onto 3D colors and then treat as 3D.  Can also define custom functions-- should take
    #      in any observation and return a new vector describing its color (in >= 1D).  also support
    #      a list of functions (one per data matrix)
    #TODO: support plotting of error bars via streamtubes and/or ribbon plots
    #TODO: support plotting "vector fields" by taking a list of two matrices of coordinats,
    #      one specifying the start points and the other specifying the end points of each
    #      vector
    
    #Write a general "hypertools function" that does all analyses by calling normalize --> align --> reduce
    #in sequence.
    
    #TODO: support for nested (>2 level dataframes)
    #TODO: support interpolation for line_styles and combo_styles
    #TODO: define default options outside of this function in a dictionary
    #TODO: normalization options: z-score down rows, across columns, both, or arbitrary functions (take
    #      in a data matrix and return a new matrix of the same type/size).  can also provide lists of
    #      normalization functions to be applied separately to each data matrix.
    #TODO: add support for multiple reduce functions (and/or reduce_kwargs)-- one per data matrix
    #TODO: add hyperalignment and SRM.  also allow different subsets of the data to be aligned, e.g.
    #      according to the unique IDs specified in a list of group IDs, some column of the data matrix,
    #      cluster IDs, or a custom function (or a dictionary of functions specifying how different
    #      group IDs should be aligned); functions should take in a list of data matrices and returned an aligned
    #      list in the same format.
    
    style = style.lower()
    
    assert style in marker_styles + line_styles + combo_styles, f'Unsupported plot style: {style}'
    
    if is_combo(style):
        mode='lines+markers'
    elif is_line(style):
        mode='lines'
    else:
        mode='markers'
    
    if is_line(style):
        if not('line' in kwargs.keys()):
                kwargs['line'] = {}
        #if not('opacity' in kwargs['line'].keys()):
        #    kwargs['line']['opacity'] = float(defaults['plot']['opacity'])
        
        dash = is_dashed(style)
        if dash:
            print(f'dashed line: {dash}')
            if not('dash' in kwargs['line'].keys()):
                kwargs['line']['dash'] = dash_styles[dash]
    if is_marker(style):
        if is_bigmarker(style):
            size = int(defaults['plot']['bigmarkersize'])
        else:
            size = int(defaults['plot']['markersize'])
        
        if not('marker' in kwargs.keys()):
            kwargs['marker'] = {}
        if not('size' in kwargs['marker'].keys()):
            if is_bigmarker(style):
                kwargs['marker']['size'] = int(defaults['plot']['bigmarkersize'])
            else:
                kwargs['marker']['size'] = int(defaults['plot']['markersize'])        
        if not('opacity' in kwargs['marker'].keys()):
            kwargs['marker']['opacity'] = float(defaults['plot']['opacity'])            
    
    names = [str(n) for n in np.array(reduced_data.index.levels[0])]
    
    if reduced_data.shape[1] == 2:
        shapes = [go.Scatter(x=d.iloc[:, 0], y=d.iloc[:, 1], mode=mode, name=names[i], **kwargs) for i, d in enumerate(split_data)]
    elif reduced_data.shape[1] == 3:
        shapes = [go.Scatter3d(x=d.iloc[:, 0], y=d.iloc[:, 1], z=d.iloc[:, 2], mode=mode, name=names[i], **kwargs) for i, d in enumerate(split_data)]
    
    if HYPERTOOLS_NOTEBOOK_MODE:
        return iplot(shapes)
    else:
        return plot(shapes)

In [113]:
x_r = reduce(xs, algorithm='UMAP')

In [114]:
plotter(xs, '.')

In [115]:
plotter(xs, style='-:')

dashed line: -:


In [116]:
plotter(xs, style='-o')

In [117]:
plotter(xs, style='.', reduce_kwargs={'ndims': 2})

In [118]:
plotter(xs, style='-', reduce_kwargs={'ndims': 2})

In [119]:
sotus_corpus = get_corpus('sotus')

In [ ]:
plotter(sotus_corpus, 'o', text_model='LatentDirichletAllocation')